# **Dataset Preparation**
This script prepares the dataset for training by performing the following operations:
1. **Dataset Directory and Class Selection:**
    * Defines the dataset directory, output directory, and the processed directory.
    * Specifies the selected animal classes to include in the project.

2. **Image Limits and Preprocessing Settings:**
    * Limits the maximum number of images per class to 650.
    * Sets the desired image size for resizing (e.g., 128x128).

3. **Output Directories Setup:**
    * Clears existing directories for the output and processed data, ensuring a clean start.
    * Recreates these directories for saving processed images.

4. **Image Copying and Processing:**
    * Iterates through the selected animal classes.
    * Copies a subset of images (up to the specified limit) for each class to the output directory.
    * Resizes and normalizes images before saving them in the processed directory.

In [2]:
# Import necessary libraries
import cv2
import numpy as np
import os
import shutil
from pathlib import Path

# Dataset directory (Adjust based on your file structure)
data_dir = "/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages"
output_dir = "/kaggle/working/selected_animals_dataset"
processed_dir = "/kaggle/working/processed_dataset"

# Selected classes
selected_classes = [
    "collie", "dolphin", "elephant", "fox", "moose", 
    "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"
]

# Maximum number of images to keep per class
max_images_per_class = 650

# Desired image size
image_size = (128, 128)  # Example size, can be adjusted based on model input

# Clear and recreate the output and processed directories
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

if os.path.exists(processed_dir):
    shutil.rmtree(processed_dir)
os.makedirs(processed_dir)

# Copy selected classes and process images
for animal_class in selected_classes:
    class_path = os.path.join(data_dir, animal_class)
    if os.path.exists(class_path):
        images = sorted(os.listdir(class_path))[:max_images_per_class]
        output_class_path = os.path.join(output_dir, animal_class)
        processed_class_path = os.path.join(processed_dir, animal_class)
        os.makedirs(output_class_path, exist_ok=True)
        os.makedirs(processed_class_path, exist_ok=True)

        for image in images:
            source_path = os.path.join(class_path, image)
            target_path = os.path.join(output_class_path, image)
            shutil.copy2(source_path, target_path)

            # Load image, resize and normalize
            img = cv2.imread(source_path)
            if img is not None:
                img_resized = cv2.resize(img, image_size)
                img_normalized = img_resized / 255.0
                processed_image_path = os.path.join(processed_class_path, image)
                cv2.imwrite(processed_image_path, (img_normalized * 255).astype(np.uint8))

print(f"Selected classes and the first {max_images_per_class} images have been copied to '{output_dir}', and processed images saved to '{processed_dir}'.")

Selected classes and the first 650 images have been copied to '/kaggle/working/selected_animals_dataset', and processed images saved to '/kaggle/working/processed_dataset'.


# **Train and Test Data Preparation**
This script processes the dataset for training and testing by performing the following operations:
1. **Dataset and Label Initialization:**
    * Defines the processed dataset directory.
    * Lists the selected animal classes and maps them to numerical labels.

2. **Data Loading and Normalization:**
    * Iterates through the processed images for each selected class.
    * Loads and normalizes the pixel values to the range [0, 1].

3. **Data Conversion:**
    * Converts the lists of images and labels into NumPy arrays for model compatibility.

4. **Data Splitting:**
    * Splits the dataset into training and testing sets with a 70-30 split.
    * Ensures reproducibility using a fixed random seed.

In [4]:
# Import necessary libraries
import os
import numpy as np
from sklearn.model_selection import train_test_split

# Processed dataset directory
processed_dir = "/kaggle/working/processed_dataset"

# Selected classes
selected_classes = [
    "collie", "dolphin", "elephant", "fox", "moose", 
    "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"
]

# Prepare data and labels
X = []  # Images
y = []  # Labels
class_mapping = {cls: idx for idx, cls in enumerate(selected_classes)}

# Load processed images and labels
for animal_class in selected_classes:
    class_path = os.path.join(processed_dir, animal_class)
    if os.path.exists(class_path):
        images = sorted(os.listdir(class_path))

        for image in images:
            source_path = os.path.join(class_path, image)

            # Load image and normalize (already resized)
            img = cv2.imread(source_path)
            if img is not None:
                img_normalized = img / 255.0
                X.append(img_normalized)
                y.append(class_mapping[animal_class])

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Data has been split into training and testing sets.\nTrain size: {len(X_train)}, Test size: {len(X_test)}")

Data has been split into training and testing sets.
Train size: 4550, Test size: 1950


# **Data Augmentation for Training Set**

1. **Data Augmentation Initialization:**
    * Creates an instance of ImageDataGenerator with the following augmentation options:
        * **Rotation:** Random rotations within ±20 degrees.
        * **Horizontal and Vertical Shifts:** Randomly shifts images horizontally and vertically by 20% of the total width or height.
        * **Shear Transformations:** Applies random shearing transformations.
        * **Zoom:** Randomly zooms into the images by 20%.
        * **Horizontal Flip:** Randomly flips images horizontally.
        * **Fill Mode:** Specifies how to fill empty pixels created after transformations.

2. **Augmentation Application::**
   * Augments a batch of training images (X_train) and corresponding labels (y_train) using a batch size of 32.

3. **Output:**
   * Confirms that data augmentation has been successfully applied to the training set.

In [5]:
# Import necessary libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Apply data augmentation to the training set
augmentation = ImageDataGenerator(
    rotation_range=20,  # Random rotation
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Shear transformations
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'  # Filling strategy for empty pixels
)

# Example: Augment a batch of images (X_train)
augmented_data = augmentation.flow(X_train, y_train, batch_size=32)

print("Data augmentation has been applied to the training set.")

Data augmentation has been applied to the training set.


# **Image Manipulation and White Balance Correction**

This script performs two key operations on image datasets:

1. **Manipulating Images**: 
   - Adjusts brightness and contrast to simulate various lighting conditions.
   - Saves the manipulated images to a specified output directory.

2. **Applying White Balance Correction**:
   - Implements the Gray World assumption to normalize colors across images.
   - Adjusts the pixel values based on the average intensities of RGB channels.

## **Image Manipulation**

### **Overview**
The function `get_manipulated_images`:
- Adjusts brightness and contrast for all images in the specified input directory.
- Saves the manipulated images to a designated output directory.

### **Key Parameters**
- **Brightness Factor**: Controls the overall brightness of the images.
- **Contrast Factor**: Adjusts the difference between dark and light areas in the image.

In [7]:
# Import necessary libraries
import cv2
import os
import numpy as np

# Function to manipulate images
def get_manipulated_images(input_dir, output_dir, brightness_factor=1.5, contrast_factor=1.2):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        output_class_path = os.path.join(output_dir, class_name)
        
        if not os.path.exists(output_class_path):
            os.makedirs(output_class_path)
        
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            
            if img is not None:
                # Apply brightness and contrast adjustments
                manipulated_img = cv2.convertScaleAbs(img, alpha=contrast_factor, beta=brightness_factor * 50)
                
                # Save manipulated image
                save_path = os.path.join(output_class_path, img_name)
                cv2.imwrite(save_path, manipulated_img)
    
    print(f"Manipulated images saved to {output_dir}")

# Paths
input_dir = "/kaggle/working/processed_dataset"
manipulated_dir = "/kaggle/working/manipulated_images"

# Generate manipulated images
get_manipulated_images(input_dir, manipulated_dir)


# Gray World Assumption implementation
def get_wb_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        output_class_path = os.path.join(output_dir, class_name)

        if not os.path.exists(output_class_path):
            os.makedirs(output_class_path)

        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)

            if img is not None:
                # Apply Gray World Assumption for white balance correction
                avg_b = np.mean(img[:, :, 0])
                avg_g = np.mean(img[:, :, 1])
                avg_r = np.mean(img[:, :, 2])
                avg_gray = (avg_b + avg_g + avg_r) / 3

                img[:, :, 0] = np.clip(img[:, :, 0] * (avg_gray / avg_b), 0, 255)
                img[:, :, 1] = np.clip(img[:, :, 1] * (avg_gray / avg_g), 0, 255)
                img[:, :, 2] = np.clip(img[:, :, 2] * (avg_gray / avg_r), 0, 255)

                # Save the corrected image
                save_path = os.path.join(output_class_path, img_name)
                cv2.imwrite(save_path, img.astype(np.uint8))

    print(f"White-balanced images saved to {output_dir}")

# Paths
wb_output_dir = "/kaggle/working/wb_corrected_images"

# Apply Gray World Assumption
get_wb_images(manipulated_dir, wb_output_dir)

Manipulated images saved to /kaggle/working/manipulated_images
White-balanced images saved to /kaggle/working/wb_corrected_images


# **Loading Manipulated and White-Balanced Datasets**

This section demonstrates how to load manipulated and white-balanced datasets for further use in training or evaluation.

In [10]:
# Paths
manipulated_dir = "/kaggle/working/manipulated_images"  # Manipulated dataset directory
wb_output_dir = "/kaggle/working/wb_corrected_images"  # White-balanced dataset directory

# Load manipulated dataset
X_manipulated_train, y_manipulated_train = load_images_from_directory(manipulated_dir, selected_classes)
print(f"Manipulated dataset loaded: {X_manipulated_train.shape}, {y_manipulated_train.shape}")

# Load white-balanced dataset
X_wb_train, y_wb_train = load_images_from_directory(wb_output_dir, selected_classes)
print(f"White-balanced dataset loaded: {X_wb_train.shape}, {y_wb_train.shape}")

Manipulated dataset loaded: (6500, 128, 128, 3), (6500,)
White-balanced dataset loaded: (6500, 128, 128, 3), (6500,)


# **Combining Datasets and Training the Model**

This section demonstrates the process of combining datasets (original, manipulated, and white-balanced) and training a CNN model on the combined data.

---

## **Combining Datasets**

### **Overview**
- Original, manipulated, and white-balanced datasets are merged into a single dataset.
- Ensures the model learns from diverse data distributions.

### **Code**
```python
# Combine Original, Manipulated, and White-Balanced Datasets
X_combined_train = np.concatenate([X_train, X_manipulated_train, X_wb_train], axis=0)
y_combined_train = np.concatenate([y_train, y_manipulated_train, y_wb_train], axis=0)


In [11]:
# Import necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from sklearn.utils import shuffle
import numpy as np

# Combine Original, Manipulated, and White-Balanced Datasets
X_combined_train = np.concatenate([X_train, X_manipulated_train, X_wb_train], axis=0)
y_combined_train = np.concatenate([y_train, y_manipulated_train, y_wb_train], axis=0)

# Convert labels to one-hot encoding
y_combined_train_one_hot = to_categorical(y_combined_train, num_classes=10)
y_test_one_hot = to_categorical(y_test, num_classes=10)

# Shuffle the combined dataset
X_combined_train, y_combined_train_one_hot = shuffle(X_combined_train, y_combined_train_one_hot, random_state=42)

# Define CNN model
model = Sequential([
    Input(shape=(128, 128, 3)),
    Conv2D(32, (3, 3)),
    LeakyReLU(negative_slope=0.1),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3)),
    LeakyReLU(negative_slope=0.1),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3)),
    LeakyReLU(negative_slope=0.1),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(256, (3, 3)),
    LeakyReLU(negative_slope=0.1),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, kernel_regularizer=l2(0.02)),
    LeakyReLU(negative_slope=0.1),
    Dropout(0.7),
    Dense(10, activation='softmax')  # 10 classes
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_combined_train, y_combined_train_one_hot,
    epochs=20,  # Increased epochs
    batch_size=32,
    validation_data=(X_test, y_test_one_hot)
)

# Model summary
model.summary()

Epoch 1/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 548s 991ms/step - accuracy: 0.2678 - loss: 7.2221 - val_accuracy: 0.4728 - val_loss: 5.4394
Epoch 2/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 556s 981ms/step - accuracy: 0.4602 - loss: 5.2359 - val_accuracy: 0.6313 - val_loss: 4.0120
Epoch 3/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 534s 972ms/step - accuracy: 0.5506 - loss: 4.0436 - val_accuracy: 0.6703 - val_loss: 3.1654
Epoch 4/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 567s 981ms/step - accuracy: 0.6025 - loss: 3.1812 - val_accuracy: 0.7031 - val_loss: 2.5174
Epoch 5/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 565s 986ms/step - accuracy: 0.6756 - loss: 2.4854 - val_accuracy: 0.7144 - val_loss: 2.1081
Epoch 6/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 534s 973ms/step - accuracy: 0.7256 - loss: 1.9903 - val_accuracy: 0.8062 - val_loss: 1.5740
Epoch 7/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 534s 972ms/step - accuracy: 0.7822 - loss: 1.6121 - val_accuracy: 0.8487 - val_loss: 1.3182
Epoch 8/20
549/549 ━━━━━━━━━━━━━━━━━━━━ 548s 998ms/step - accuracy: 0.8291 -

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 126, 126, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 126, 126, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 61, 61, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 61, 61, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 28, 28, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 12, 12, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 12, 12, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 9216)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       1,179,776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 4,712,288 (17.98 MB)

 Trainable params: 1,570,442 (5.99 MB)

 Non-trainable params: 960 (3.75 KB)

 Optimizer params: 3,140,886 (11.98 MB)

# **Evaluating the Model on Manipulated Test Set**

## **Overview**
This section evaluates the model's performance on the manipulated test set, which contains images with brightness and contrast adjustments. The goal is to assess how well the model generalizes to altered lighting conditions.


## **Steps**

### **1. Load Manipulated Test Set**
- The manipulated dataset is loaded from the specified directory.
- Images and labels are extracted.

#### **Code**
```python
manipulated_dir = "/kaggle/working/manipulated_images"  # Adjust path as needed

# Load manipulated test images and labels
X_manipulated_test, y_manipulated_test = load_images_from_directory(manipulated_dir, selected_classes)
print(f"Manipulated Test Set Loaded: {X_manipulated_test.shape}, {y_manipulated_test.shape}")

In [13]:
# Load manipulated test set
manipulated_dir = "/kaggle/working/manipulated_images"  # Adjust path as needed

# Load manipulated test images and labels
X_manipulated_test, y_manipulated_test = load_images_from_directory(manipulated_dir, selected_classes)

print(f"Manipulated Test Set Loaded: {X_manipulated_test.shape}, {y_manipulated_test.shape}")

# Convert manipulated test labels to one-hot encoding
y_manipulated_test_one_hot = to_categorical(y_manipulated_test, num_classes=10)

# Evaluate the model on manipulated test set
manipulated_loss, manipulated_accuracy = model.evaluate(X_manipulated_test, y_manipulated_test_one_hot, verbose=2)
print(f"Manipulated Test Loss: {manipulated_loss}, Manipulated Test Accuracy: {manipulated_accuracy}")

Manipulated Test Set Loaded: (6500, 128, 128, 3), (6500,)
204/204 - 40s - 198ms/step - accuracy: 0.9895 - loss: 0.3466
Manipulated Test Loss: 0.3465944528579712, Manipulated Test Accuracy: 0.9895384907722473


# **Evaluating the Model on White-Balanced Test Set**

## **Overview**
This section evaluates the model's performance on the white-balanced test set, which has been processed using the Gray World assumption for color normalization. The goal is to determine if white balance correction improves classification accuracy.

## **Steps**

### **1. Load White-Balanced Test Set**
- The white-balanced dataset is loaded from the specified directory.
- Images and labels are extracted.

#### **Code**
```python
wb_output_dir = "/kaggle/working/wb_corrected_images"  # Adjust path as needed

# Load white-balanced test images and labels
X_wb_test, y_wb_test = load_images_from_directory(wb_output_dir, selected_classes)
print(f"White-Balanced Test Set Loaded: {X_wb_test.shape}, {y_wb_test.shape}")


In [15]:
# Load white-balanced test set
wb_output_dir = "/kaggle/working/wb_corrected_images"  # Adjust path as needed

# Load white-balanced test images and labels
X_wb_test, y_wb_test = load_images_from_directory(wb_output_dir, selected_classes)

print(f"White-Balanced Test Set Loaded: {X_wb_test.shape}, {y_wb_test.shape}")

y_wb_test_one_hot = to_categorical(y_wb_test, num_classes=10)

wb_loss, wb_accuracy = model.evaluate(X_wb_test, y_wb_test_one_hot, verbose=2)
print(f"White-Balanced Test Loss: {wb_loss}, White-Balanced Test Accuracy: {wb_accuracy}")

White-Balanced Test Set Loaded: (6500, 128, 128, 3), (6500,)
204/204 - 40s - 195ms/step - accuracy: 0.9966 - loss: 0.3308
White-Balanced Test Loss: 0.33081671595573425, White-Balanced Test Accuracy: 0.9966154098510742


# **Evaluating the Model on Original Test Set**

## **Overview**
In this step, we evaluate the model's performance on the original (unaltered) test dataset to establish a baseline performance.


## **Steps**

### **1. Convert Labels to One-Hot Encoding**
- Labels are converted into a one-hot encoded format for compatibility with the categorical classification model.

#### **Code**
```python
# Convert original test labels to one-hot encoding
y_test_one_hot = to_categorical(y_test, num_classes=10)


In [16]:
# Convert original test labels to one-hot encoding
y_test_one_hot = to_categorical(y_test, num_classes=10)

# Evaluate the model on the original test set
original_loss, original_accuracy = model.evaluate(X_test, y_test_one_hot, verbose=2)

# Print the results
print(f"Original Test Loss: {original_loss}, Original Test Accuracy: {original_accuracy}")

61/61 - 12s - 195ms/step - accuracy: 0.9492 - loss: 0.4589
Original Test Loss: 0.4589402675628662, Original Test Accuracy: 0.9492307901382446


# **Comparison and Reporting of Test Set Performances**

## **Objective**
The aim was to evaluate the model's performance on three different test datasets:
1. **Original Test Set**
2. **Manipulated Test Set**
3. **White-Balanced Test Set**


## **Results Summary**
| Test Set                  | Accuracy   | Loss    |
|---------------------------|------------|---------|
| **Original Test Set**     | **94.92%** | 0.4589  |
| **Manipulated Test Set**  | **98.95%** | 0.3466  |
| **White-Balanced Test Set** | **99.66%** | 0.3308  |


## **Insights**
1. **Original Test Set**:
   - The model performs well on the original dataset, achieving an accuracy of **94.92%**.
   - This demonstrates the model's ability to correctly classify data from its training distribution.

2. **Manipulated Test Set**:
   - With brightness and contrast adjustments, the model achieves an improved accuracy of **98.95%**.
   - This suggests that the model has learned robust features and generalizes well to visually altered data.

3. **White-Balanced Test Set**:
   - Applying the Gray World assumption for white balance correction further improves accuracy to **99.66%**.
   - This shows the effectiveness of normalization in enhancing the model's interpretability of image features.

## **Suggestions for Improvement**
### **What Could Have Been Done Differently**
Since I (or we) developed this pipeline, here are a few areas where alternative approaches could have been explored:

1. **Model Enhancements**:
   - Using pretrained models such as ResNet or EfficientNet could have sped up training and improved results further.
   - Adding attention mechanisms like SE blocks or CBAM to the model could have refined feature extraction.

2. **Data Preparation**:
   - Applying more diverse augmentation techniques, such as random noise injection or affine transformations, could have simulated real-world variability better.
   - Testing with other color correction algorithms (e.g., Histogram Equalization) might have provided additional insights.

3. **Hyperparameter Optimization**:
   - Performing systematic hyperparameter tuning with libraries like Optuna or HyperOpt might have yielded better configurations.
   - Experimenting with optimizers like SGD with momentum or adaptive optimizers (e.g., AdamW) could have further fine-tuned performance.

4. **Evaluation**:
   - Cross-validation could have been used to ensure that the results were consistent across different splits of the dataset.
   - Testing the model on entirely unseen datasets could have provided a better understanding of its real-world applicability.

### **If the Scores Were Low (<50%)**
Had the accuracy been significantly lower, here are steps that could have been taken:
- **Model Simplification**: Reducing overfitting by simplifying the architecture or adding dropout layers.
- **Augmentation**: Introducing more aggressive augmentations to combat overfitting and improve generalization.
- **Fine-Tuning**: Training on a smaller dataset using transfer learning could have helped establish a stronger baseline.

## **Conclusion**
The model shows exceptional generalization, achieving high accuracy across all test sets. However, improvements in architecture, data preprocessing, and hyperparameter tuning could have potentially pushed the performance even further. The white balance correction proved to be an impactful enhancement, significantly boosting test accuracy. This experiment lays a solid foundation for future iterations of similar tasks.
